In [3]:
import json
import numpy as np
from GRID import GRID
import matplotlib.pyplot as plt
import sys

%matplotlib inline
%config InlineBackend.figure_format = 'svg'


In [4]:

it = 1000

Eq = "Original Equation"
#Eq = 'New Equation'
#Eq = 'Transformed Variables'

var = 'Vx' #Vx Vz
name = 3

# if len( sys.argv ) > 1:
# 	it = int( sys.argv[1] )
# 	var = str( sys.argv[2] )



jsonsFile = open( "../paramsDir/paramsCGFDM3D_fp32.json" )
# jsonsFile = open( "./paramsDir/paramsCGFDM3D.json" )
params = json.load( jsonsFile )
station = json.load( open("../stationsDir/station.json") )
sta_dir = station["station(point)"]
sta_len = len(sta_dir)
print(sta_len)
sta_X = [sta_dir[str(i)][0] for i in range(sta_len)]
sta_Z = [sta_dir[str(i)][2] for i in range(sta_len)]
print(sta_X)
print(sta_Z)

StationLabels = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 
				 'H', 'I', 'J', 'K', 'L', 'M', 'N',
				 'O', 'P', 'Q',		 'R', 'S', 'T',
				 'U', 'V', 'W', 	 'X', 'Y', 'Z']


In [5]:

grid = GRID( params )

sample = 1

outputPath = params["out"]
if name == 3:
	outputPath = 'output/CGFDM3D_fp32'

fileNameX = "../" + params["out"] + "/coordX"
fileNameY = "../" + params["out"] + "/coordY"
fileNameZ = "../" + params["out"] + "/coordZ"

DT = params["DT"]

fileName = "../" + outputPath + "/%s_%d"%( var, it )
#fileName = params["out"] + "/Vs"
#fileName = "output_homo_original" +"/%s_%d"%( var, it )
print( "Draw " + fileName  )


FAST_AXIS = params["FAST_AXIS"]

srX = params["sourceX"]
srZ = params["sourceZ"]
DH = params["DH"]
locX = srX * DH
locZ = srZ * DH


'''
for Z in range( grid.PZ ):
	for Y in range( grid.PY ):
		for X in range( grid.PX ):
			print( "nx = %d, ny = %d, nz = %d\n" % ( grid.nx[X], grid.ny[Y], grid.nz[Z] ) )
'''

sliceX = params["sliceX"] - grid.frontNX
sliceY = params["sliceY"] - grid.frontNY
sliceZ = params["sliceZ"] - grid.frontNZ

for mpiSliceX in range( grid.PX ):
	if sliceX[mpiSliceX] >= 0 and sliceX[mpiSliceX] < grid.nx[mpiSliceX]:
		break

for mpiSliceY in range( grid.PY ):
	if sliceY[mpiSliceY] >= 0 and sliceY[mpiSliceY] < grid.ny[mpiSliceY]:
		break

for mpiSliceZ in range( grid.PZ ):
	if sliceZ[mpiSliceZ] >= 0 and sliceZ[mpiSliceZ] < grid.nz[mpiSliceZ]:
		break



if FAST_AXIS == 'Z':
	dataX = np.zeros( [grid.NX, grid.NZ] )
	dataZ = np.zeros( [grid.NX, grid.NZ] )
	data2 = np.zeros( [grid.NX, grid.NZ] )
else:
	dataX = np.zeros( [grid.NZ, grid.NX] )
	dataZ = np.zeros( [grid.NZ, grid.NX] )
	data2 = np.zeros( [grid.NZ, grid.NX] )


mpiY = mpiSliceY
for mpiZ in range( grid.PZ ):
	for mpiX in range( grid.PX ):
		fileX = open( "%s_Y_mpi_%d_%d_%d.bin" % ( fileNameX, mpiX, mpiY, mpiZ ), "rb" )
		fileZ = open( "%s_Y_mpi_%d_%d_%d.bin" % ( fileNameZ, mpiX, mpiY, mpiZ ), "rb" )
		file  = open( "%s_Y_mpi_%d_%d_%d.bin" % ( fileName , mpiX, mpiY, mpiZ ), "rb" )
		print( fileName )
		nx = grid.nx[mpiX]
		nz = grid.nz[mpiZ]
		print( "nx = %d, nz = %d" % ( nx, nz ) )
		datax = np.fromfile( fileX, dtype='float32', count = nx * nz )
		#print( np.shape( datax ) )
		dataz = np.fromfile( fileZ, dtype='float32', count = nx * nz )
		data_  = np.fromfile( file, dtype='float32', count = nx * nz )
		I  = grid.frontNX[mpiX]
		I_ = grid.frontNX[mpiX] + nx
		K  = grid.frontNZ[mpiZ]
		K_ = grid.frontNZ[mpiZ] + nz

		if FAST_AXIS == 'Z':
			dataX[I:I_, K:K_] = np.reshape( datax, ( nx, nz ) )
			dataZ[I:I_, K:K_] = np.reshape( dataz, ( nx, nz ) )
			data2[I:I_, K:K_] = np.reshape( data_, ( nx, nz ) )
		else:
			dataX[K:K_, I:I_] = np.reshape( datax, ( nz, nx ) )
			dataZ[K:K_, I:I_] = np.reshape( dataz, ( nz, nx ) )
			data2[K:K_, I:I_] = np.reshape( data_, ( nz, nx ) )


#data = data / C
dpi = 500
#fig = plt.figure( dpi = dpi, figsize = ( 1920 // dpi, 1080 // dpi ) )
unit = 1000 # 1km = 1000m
vm = np.max( np.abs( data2 ) )
dataX = dataX/unit
dataZ = dataZ/unit
#plt.pcolormesh( dataX, dataZ, data, cmap = "jet" )

plt.figure(dpi=dpi,figsize=(8,6))
plt.plot( dataX[:, -1], dataZ[:, -1],  'k', linewidth = 2  )
plt.plot( dataX[:,  0], dataZ[:,  0],  'k', linewidth = 2  )
plt.plot( dataX[-1, :], dataZ[-1, :],  'k', linewidth = 2  )
plt.plot( dataX[ 0, :], dataZ[ 0, :],  'k', linewidth = 2  )

plt.plot( dataX[srZ, srX], dataZ[srZ, srX], "*", color="r", ms=10 )
# plt.plot( dataX[sta_Z, sta_X], dataZ[sta_Z, sta_X], "v", color="k", ms=5 )
# for i in range(sta_len):
# 	plt.text( dataX[sta_Z[i], sta_X[i]], dataZ[sta_Z[i], sta_X[i]]+0.4, StationLabels[i], color="k", ha='center', va='bottom')

plt.pcolormesh( dataX, dataZ, data2, vmax = vm, vmin = -vm, cmap = "seismic" )
#plt.pcolormesh( data[::sample, ::sample] // unit, vmin = -vm / 2, vmax = vm, cmap = "jet" )
#plt.pcolormesh( data[5:grid.NZ -5:sample, 5:grid.NX - 5:sample] // unit, cmap = "seismic" )
#plt.pcolormesh( data, vmax = vm, vmin = -vm, cmap = "jet" )
#plt.imshow( data, cmap = "jet", origin= "lower" )

cb = plt.colorbar( format='%.0e', shrink=0.7)
cbRange = np.linspace( -vm, vm, 5 )
cb.set_ticks( cbRange )
#cb.ax.tick_params(labelsize=14)
#plt.colorbar( orientation = "horizontal" )
plt.axis( "image" )
#fileName = "%s: t = %.3fs" % (var, it * DT )
#fileName = "%s: %s" % ( Eq, var )

if Eq == 'Original Equation':
	if var == 'Txz':
		fileName = "%s: $\sigma_{xz}$" % Eq
	if var == 'Vx':
		fileName = "%s: $v_x$" % Eq
	if var == 'Vz':
		fileName = "%s: $v_z$" % Eq


if Eq == 'New Equation':
	if var == 'Txz':
		fileName = "%s: $\Sigma_{xz}$" % Eq
	if var == 'Vx':
		fileName = "%s: $V_x$" % Eq
	if var == 'Vz':
		fileName = "%s: $V_z$" % Eq



if Eq == 'Transformed Variables':
	if var == 'Txz':
		fileName = "%s: $\Sigma_{xz}/C_v$" % Eq
	if var == 'Vx':
		fileName = "%s: $V_x/C_v$" % Eq
	if var == 'Vz':
		fileName = "%s: $V_z/C_v$" % Eq


Xmin = np.min( dataX )
Xmax = np.max( dataX )
Zmin = np.min( dataZ )
Zmax = np.max( dataZ )
xticksRange = np.linspace( -Xmax, Xmax, 5)
 
zticksRange = np.linspace( Zmin, Zmax, 5 )

plt.xticks(xticksRange)
plt.yticks(zticksRange)

plt.tick_params(axis='both',which='major')

# plt.ylabel( "Vertical Distance-Z (km)" )
# plt.xlabel( "Horizontal Distance-X (km)" )
plt.ylabel( "Z-axis (km)" )
plt.xlabel( "X-axis (km)" )

if name == 0:
	fileName = 'CGFDM3D' 
if name == 1:
	fileName = 'CGFDM3D-LSRK' 
if name == 2:
	fileName = 'CGFDM3D-CJMVS' 
if name == 3:
	fileName = 'CGFDM3D-float' 


# plt.title( fileName + ':' + var, fontsize = 14 )
if (var == "Vx"):
	strnm = "$v_x$ (m/s)"
elif (var == "Vy"):
	strnm = "$v_y$ (m/s)"
elif (var == "Vz"):
	strnm = "$v_z$ (m/s)"
plt.title( 'CGFDM-FP32' + ':' + strnm, fontsize = 14 )

#plt.savefig( varname + Eq + ".pdf" )
# plt.savefig( fileName + var + ".pdf" )
plt.savefig( "./imagesDir/" + fileName + var + "_fp32.pdf" )		# src_fp16
#plt.plot( dataZ[grid.NZ - 1, :] )

#print( grid )

In [6]:
#!/usr/bin/env python
'''
Author: Wenqiang Wang @ SUSTech on Sep 11, 2021
15:40
'''
import json
import numpy as np
from GRID import GRID
import matplotlib.pyplot as plt
import sys

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

it = 1000

Eq = "Original Equation"
#Eq = 'New Equation'
#Eq = 'Transformed Variables'

var = 'Vx' #Vx Vz
name = 3


# if len( sys.argv ) > 1:
# 	it = int( sys.argv[1] )
# 	var = str( sys.argv[2] )



jsonsFile = open( "../paramsDir/paramsCGFDM3D_half2.json" )
params = json.load( jsonsFile )
grid = GRID( params )

sample = 1

outputPath = params["out"]
if name == 3:
	outputPath = 'output/CGFDM3D_half2'

fileNameX = "../" + params["out"] + "/coordX"
fileNameY = "../" + params["out"] + "/coordY"
fileNameZ = "../" + params["out"] + "/coordZ"

DT = params["DT"]

varname = "./png_theorem/%s_%d" % ( var, it )

fileName = "../" + outputPath + "/%s_%d"%( var, it )
#fileName = params["out"] + "/Vs"
#fileName = "output_homo_original" +"/%s_%d"%( var, it )
print( "Draw " + fileName  )


FAST_AXIS = params["FAST_AXIS"]


'''
for Z in range( grid.PZ ):
	for Y in range( grid.PY ):
		for X in range( grid.PX ):
			print( "nx = %d, ny = %d, nz = %d\n" % ( grid.nx[X], grid.ny[Y], grid.nz[Z] ) )
'''

sliceX = params["sliceX"] - grid.frontNX
sliceY = params["sliceY"] - grid.frontNY
sliceZ = params["sliceZ"] - grid.frontNZ

for mpiSliceX in range( grid.PX ):
	if sliceX[mpiSliceX] >= 0 and sliceX[mpiSliceX] < grid.nx[mpiSliceX]:
		break

for mpiSliceY in range( grid.PY ):
	if sliceY[mpiSliceY] >= 0 and sliceY[mpiSliceY] < grid.ny[mpiSliceY]:
		break

for mpiSliceZ in range( grid.PZ ):
	if sliceZ[mpiSliceZ] >= 0 and sliceZ[mpiSliceZ] < grid.nz[mpiSliceZ]:
		break



if FAST_AXIS == 'Z':
	dataX = np.zeros( [grid.NX, grid.NZ] )
	dataZ = np.zeros( [grid.NX, grid.NZ] )
	data1  = np.zeros( [grid.NX, grid.NZ] )
else:
	dataX = np.zeros( [grid.NZ, grid.NX] )
	dataZ = np.zeros( [grid.NZ, grid.NX] )
	data1  = np.zeros( [grid.NZ, grid.NX] )


mpiY = mpiSliceY
for mpiZ in range( grid.PZ ):
	for mpiX in range( grid.PX ):
		fileX = open( "%s_Y_mpi_%d_%d_%d.bin" % ( fileNameX, mpiX, mpiY, mpiZ ), "rb" )
		fileZ = open( "%s_Y_mpi_%d_%d_%d.bin" % ( fileNameZ, mpiX, mpiY, mpiZ ), "rb" )
		file  = open( "%s_Y_mpi_%d_%d_%d.bin" % ( fileName , mpiX, mpiY, mpiZ ), "rb" )
		print( fileName )
		nx = grid.nx[mpiX]
		nz = grid.nz[mpiZ]
		print( "nx = %d, nz = %d" % ( nx, nz ) )
		datax = np.fromfile( fileX, dtype='float32', count = nx * nz )
		#print( np.shape( datax ) )
		dataz = np.fromfile( fileZ, dtype='float32', count = nx * nz )
		data_  = np.fromfile( file, dtype='float32', count = nx * nz )
		I  = grid.frontNX[mpiX]
		I_ = grid.frontNX[mpiX] + nx
		K  = grid.frontNZ[mpiZ]
		K_ = grid.frontNZ[mpiZ] + nz

		if FAST_AXIS == 'Z':
			dataX[I:I_, K:K_] = np.reshape( datax, ( nx, nz ) )
			dataZ[I:I_, K:K_] = np.reshape( dataz, ( nx, nz ) )
			data1[I:I_, K:K_] = np.reshape( data_, ( nx, nz ) )
		else:
			dataX[K:K_, I:I_] = np.reshape( datax, ( nz, nx ) )
			dataZ[K:K_, I:I_] = np.reshape( dataz, ( nz, nx ) )
			data1[K:K_, I:I_] = np.reshape( data_, ( nz, nx ) )


#data = data / C
dpi = 500
#fig = plt.figure( dpi = dpi, figsize = ( 1920 // dpi, 1080 // dpi ) )
unit = 1000 # 1km = 1000m
# vm = np.max( np.abs( data1 ) )
dataX = dataX/unit
dataZ = dataZ/unit
#plt.pcolormesh( dataX, dataZ, data, cmap = "jet" )

plt.figure(dpi=dpi,figsize=(8,6))
plt.plot( dataX[:, -1], dataZ[:, -1],  'k', linewidth = 2  )
plt.plot( dataX[:,  0], dataZ[:,  0],  'k', linewidth = 2  )
plt.plot( dataX[-1, :], dataZ[-1, :],  'k', linewidth = 2  )
plt.plot( dataX[ 0, :], dataZ[ 0, :],  'k', linewidth = 2  )

plt.plot( dataX[srZ, srX], dataZ[srZ, srX], "*", color="r", ms=10 )
# plt.plot( dataX[sta_Z, sta_X], dataZ[sta_Z, sta_X], "v", color="k", ms=5 )
# for i in range(sta_len):
# 	plt.text( dataX[sta_Z[i], sta_X[i]], dataZ[sta_Z[i], sta_X[i]]+0.4, StationLabels[i], color="k", ha='center', va='bottom')

plt.pcolormesh( dataX, dataZ, data1, vmax = vm, vmin = -vm, cmap = "seismic" )
#plt.pcolormesh( data[::sample, ::sample] // unit, vmin = -vm / 2, vmax = vm, cmap = "jet" )
#plt.pcolormesh( data[5:grid.NZ -5:sample, 5:grid.NX - 5:sample] // unit, cmap = "seismic" )
#plt.pcolormesh( data, vmax = vm, vmin = -vm, cmap = "jet" )
#plt.imshow( data, cmap = "jet", origin= "lower" )

cb = plt.colorbar( format='%.0e', shrink=0.7)
cbRange = np.linspace( -vm, vm, 5 )
cb.set_ticks( cbRange )
#cb.ax.tick_params(labelsize=14)
#plt.colorbar( orientation = "horizontal" )
plt.axis( "image" )
#fileName = "%s: t = %.3fs" % (var, it * DT )
#fileName = "%s: %s" % ( Eq, var )

if Eq == 'Original Equation':
	if var == 'Txz':
		fileName = "%s: $\sigma_{xz}$" % Eq
	if var == 'Vx':
		fileName = "%s: $v_x$" % Eq
	if var == 'Vz':
		fileName = "%s: $v_z$" % Eq


if Eq == 'New Equation':
	if var == 'Txz':
		fileName = "%s: $\Sigma_{xz}$" % Eq
	if var == 'Vx':
		fileName = "%s: $V_x$" % Eq
	if var == 'Vz':
		fileName = "%s: $V_z$" % Eq



if Eq == 'Transformed Variables':
	if var == 'Txz':
		fileName = "%s: $\Sigma_{xz}/C_v$" % Eq
	if var == 'Vx':
		fileName = "%s: $V_x/C_v$" % Eq
	if var == 'Vz':
		fileName = "%s: $V_z/C_v$" % Eq


Xmin = np.min( dataX )
Xmax = np.max( dataX )
Zmin = np.min( dataZ )
Zmax = np.max( dataZ )
xticksRange = np.linspace( -Xmax, Xmax, 5)
 
zticksRange = np.linspace( Zmin, Zmax, 5 )

plt.xticks(xticksRange)
plt.yticks(zticksRange)

plt.tick_params(axis='both',which='major')

# plt.ylabel( "Vertical Distance-Z (km)" )
# plt.xlabel( "Horizontal Distance-X (km)" )
plt.ylabel( "Z-axis (km)" )
plt.xlabel( "X-axis (km)" )


if name == 0:
	fileName = 'CGFDM3D' 
if name == 1:
	fileName = 'CGFDM3D-LSRK' 
if name == 2:
	fileName = 'CGFDM3D-CJMVS' 
if name == 3:
	fileName = 'CGFDM3D-half2' 


# plt.title( fileName + ':' + var, fontsize = 14 )
if (var == "Vx"):
	strnm = "$v_x$ (m/s)"
elif (var == "Vy"):
	strnm = "$v_y$ (m/s)"
elif (var == "Vz"):
	strnm = "$v_z$ (m/s)"
plt.title( 'CGFDM-FP16' + ':' + strnm, fontsize = 14 )

#plt.savefig( varname + Eq + ".pdf" )
# plt.savefig( fileName + var + ".png" )
plt.savefig( "./imagesDir/" + fileName + var + "_fp16.pdf" )		# src_fp16
#plt.plot( dataZ[grid.NZ - 1, :] )

#print( grid )

In [7]:
from matplotlib.ticker import FuncFormatter

plt.figure(dpi=dpi,figsize=(8,6))
plt.plot( dataX[:, -1], dataZ[:, -1],  'k', linewidth = 2  )
plt.plot( dataX[:,  0], dataZ[:,  0],  'k', linewidth = 2  )
plt.plot( dataX[-1, :], dataZ[-1, :],  'k', linewidth = 2  )
plt.plot( dataX[ 0, :], dataZ[ 0, :],  'k', linewidth = 2  )

error = data2-data1
error = np.abs(error) / np.max(np.abs(data2))
# error = np.abs(error)
# vm = np.max( np.abs( error ) )
vm = 0.01
im = plt.pcolormesh( dataX, dataZ, error, vmax = vm, cmap = "RdPu" )
# plt.pcolormesh( dataX, dataZ, error, cmap = "seismic" )

# cb = plt.colorbar( format='%.4f', shrink=0.75)
cb = plt.colorbar(im, shrink=0.75, format=FuncFormatter(lambda val, _: f'{val*100:.2f}%'))

cbRange = np.linspace( 0, vm, 5 )
cb.set_ticks( cbRange )
#cb.ax.tick_params(labelsize=14)
#plt.colorbar( orientation = "horizontal" )
plt.axis( "image" )
#fileName = "%s: t = %.3fs" % (var, it * DT )
#fileName = "%s: %s" % ( Eq, var )

Xmin = np.min( dataX )
Xmax = np.max( dataX )
Zmin = np.min( dataZ )
Zmax = np.max( dataZ )
xticksRange = np.linspace( -Xmax, Xmax, 5)
 
zticksRange = np.linspace( Zmin, Zmax, 5 )

plt.xticks(xticksRange)
plt.yticks(zticksRange)

plt.tick_params(axis='both',which='major')

# plt.ylabel( "Vertical Distance-Z (km)" )
# plt.xlabel( "Horizontal Distance-X (km)" )
plt.ylabel( "Z-axis (km)" )
plt.xlabel( "X-axis (km)" )

if name == 0:
	fileName = 'CGFDM3D' 
if name == 1:
	fileName = 'CGFDM3D-LSRK' 
if name == 2:
	fileName = 'CGFDM3D-CJMVS' 
if name == 3:
	fileName = 'CGFDM3D-res' 


# plt.title( fileName + ':' + var, fontsize = 14 )
if (var == "Vx"):
	strnm = "$v_x$ (m/s)"
elif (var == "Vy"):
	strnm = "$v_y$ (m/s)"
elif (var == "Vz"):
	strnm = "$v_z$ (m/s)"
# plt.title( 'CGFDM-absolute error ' + "(m/s)", fontsize = 14 )
plt.title( 'CGFDM-relative error ', fontsize = 14 )

#plt.savefig( varname + Eq + ".pdf" )
# plt.savefig( fileName + var + ".pdf" )

plt.savefig( "./imagesDir/" + fileName + var + "_error.pdf" )		# src_fp16
#plt.plot( dataZ[grid.NZ - 1, :] )

#print( grid )